# This Notebook contains the analysis of the behaviour of the different pixels on one subdie
all Data is taken at 25C

In [ ]:
######same configuration, different measurements#

print("starting subdie pixel differences analysis...")
import custom_analyzer as canal
import time
import matplotlib.pyplot as plt
import custom_pathlibrary as cpath
import numpy as np
import math
import os

folder = "figs/subdie_pixel_differences/"

path = cpath.PLib()
identifier = path.dieNr_to_identifier

files = canal.get_all_filenames(startdir= "data_storage/")
dataset = canal.Dataset()
dataset.add_files(files=files, only_measurements=True)

for i in identifier:
    to_stdout= False
    close_fig = True
    fig= None

    plt.ion()
    filename = f"{i}_subdie_pixel_differences.png"
    if not os.path.isdir(str(folder)):
        os.makedirs(str(folder))
    else:
        pass

    if fig is None:
        fig = plt.figure(figsize=[16, 9])

    # plotting main data (optional)
    for diode_ID in ["1,1", "1,2", "1,3", "1,4", "2,1", "2,2", "2,3", "2,4", "3,1", "3,2", "3,3", "3,4",
            "4,1", "4,2", "4,3", "4,4"]:
            try:
                slicename, slice = dataset.filter(slicename=diode_ID, parameter="identifier", value=i)
                dataset.filter_slice(target_slice_name=slicename, parameter="diode_Nr", value=diode_ID)
                dataset.filter_slice(target_slice_name=slicename, parameter="temp_at_end", value=25)
                dataset.clean_slice_data_rbias(target_slice_name=slicename, verbose=True)
                data = dataset.get_slice_mean(target_slice_name=slicename, rbias_only=True)
                x = []
                y = []  
                for key, value in data.items():
                    x.append(key)
                    y.append(value)
                if len(x) > 0:
                    data = dataset.get_slice_se(target_slice_name=slicename, rbias_only=True)
                    err_y = []
                    x2 = []
                    for key, value in data.items():
                        x2.append(key)
                        err_y.append(value)
                    median  = dataset.get_slice_median(target_slice_name=slicename, rbias_only=True)
                    med_y = []
                    x3 = []
                    for key, value in data.items():
                        x3.append(key)
                        med_y.append(value)
                    mode  = dataset.get_slice_mode(target_slice_name=slicename, rbias_only=True)
                    mode_y = []
                    x4 = []
                    for key, value in data.items():
                        x4.append(key)
                        mode_y.append(value)
                    if diode_ID.startswith("3") or diode_ID.startswith("4"):
                        settings = "--"
                    else:
                        settings = "*-"

                    plt.errorbar(x, y, yerr=err_y, fmt=settings, capsize=5, label=diode_ID, capthick=0.5, markevery=5)
                    plt.plot(x3, med_y, ":", label=f"{diode_ID} median", markersize=5, alpha=0.5)
                    #plt.plot(x4, mode_y, "-.", label=f"{diode_ID} mode", markersize=5, alpha=0.5)
                    #plt.errorbar(x, y, fmt=settings, capsize=5, label=i, capthick=0.5, markevery=5)
                    print(f"Processed slice {i} with {len(x)} points")
            except Exception as e:
                print(f"Error processing slice {i}: {e}")
                continue

    plt.legend()
    plt.legend(fontsize=10, loc="best", ncols=8)#, bbox_to_anchor=(0.6, -0.1))

    plt.title(f"identifier {i} - Subdie Pixel Differences", fontsize=20)
    plt.yscale("log")
    plt.xscale("linear")
    #plt.xlim(left=40, right=48)
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')   
    plt.xlabel("Voltage (V)")
    plt.ylabel("Current (A)")
    plt.grid(True)
    fig.set_facecolor('w')
    if to_stdout:
        plt.show()
    else:
        fig.savefig(os.path.join(folder, filename) if folder else filename, bbox_inches='tight', dpi=500)
        if close_fig:
            plt.close(fig)
        print(f"Graph saved to {os.path.join(folder, filename) if folder else filename}")

starting subdie pixel differences analysis...
len target_slice 38
Cleaning slice 1,1 with 38 entries
len cleaned_t_s 38
Yield of cleaned slice 1,1: 100.00%
Processed slice 04E10 with 101 points
len target_slice 38
Cleaning slice 1,2 with 38 entries
Removing data_storage/Wafer_at_2025_06_16-18_35_06/Die_35/Subdie_1_ADiodes_16/IV_Die_35_Subdie_1_Diode_1,2_measurement_1.json from slice 1,2 due to outliers in rbias data
Removing data_storage/Wafer_at_2025_06_16-18_35_06/Die_24/Subdie_1_ADiodes_16/IV_Die_24_Subdie_1_Diode_1,2.json from slice 1,2 due to outliers in rbias data
len cleaned_t_s 36
Yield of cleaned slice 1,2: 94.74%
Processed slice 04E10 with 101 points
len target_slice 38
Cleaning slice 1,3 with 38 entries
Removing data_storage/Wafer_at_2025_06_16-18_35_06/Die_35/Subdie_1_ADiodes_16/IV_Die_35_Subdie_1_Diode_1,3_measurement_1.json from slice 1,3 due to outliers in rbias data
Removing data_storage/Wafer_at_2025_06_16-18_35_06/Die_25/Subdie_1_ADiodes_16/IV_Die_25_Subdie_1_Diode_1,

## in order to improve the filtering algo, we analyze the 1st derivative of the IV curves


In [ ]:
### single measurement with derivative



import custom_analyzer as canal
import time
import matplotlib.pyplot as plt
import custom_pathlibrary as cpath
import numpy as np
import math
import json
import os
files = canal.get_all_filenames(startdir= "data_storage/")
folder = "figs/all_with_dervs/"
if not os.path.isdir(str(folder)):
    os.makedirs(str(folder))
def get_gr_1st_derivative(x ,y):
    xd = []
    yd = []
    for i in range(len(x)-1):
        xd.append((x[i]+x[i+1])/2)
        yd.append((y[i+1]-y[i])/1.)
    return xd, yd


for i in files:
    try:
        if not os.path.isfile(i):
            raise FileNotFoundError(f"File {i} does not exist.")
        if i.endswith(".json") and "IV_" in i:
            with open(i, 'r', encoding='utf-8') as file:
                data = json.load(file)
            identifier = data["identifier"]
            if "02" in identifier and data["temp_at_end"] == 25:
                rbias_temp = data["iv_data_rbias"]
                rbias_data = {}
                x = []
                y = []
                for p, q in rbias_temp:
                    rbias_data[p] = q
                    x.append(p)
                    y.append(q)
                m, n = get_gr_1st_derivative(x, y)
                o, p = get_gr_1st_derivative(m, n)
                i2 = i.replace("/", "+")
                filename = f"{identifier}_{data['die_nr']}_{data['diode_Nr']}_{data['time_at_start']}_{i2}.png"
                ## plotting
                plt.ion()
                fig = plt.figure(figsize=[16, 9])
                plt.plot(x, y, label=f"{identifier} {data['die_nr']} {data['diode_Nr']}", marker='o', markersize=5, linestyle='-', alpha=0.7)
                plt.plot(m, n, label=f"derivative_1")
                plt.plot(o, p, label=f"derivative_2")

                plt.legend()
                plt.legend(fontsize=10, loc="best", ncols=8)#, bbox_to_anchor=(0.6, -0.1))

                plt.title(f"identifier {i} - Subdie Pixel Differences", fontsize=20)
                plt.yscale("linear")
                plt.xscale("linear")
                plt.axhline(0, color='black')
                plt.axvline(0, color='black')   
                plt.xlabel("Voltage (V)")
                plt.ylabel("Current (A)")
                plt.grid(True)
                fig.set_facecolor('w')
                fig.savefig(os.path.join(folder, filename) if folder else filename, bbox_inches='tight', dpi=500)
                plt.close(fig)
                print(f"Graph saved to {os.path.join(folder, filename) if folder else filename}")
    except Exception as e:
        print(f"Error processing file {i}: {e}")
        #time.sleep(1)
        continue